In [ ]:
from src.distance_measures import analyse_video, analyse_image
from src.gaussian_noise import generate_noise
from src.preprocessing import process_image, process_video
import numpy as np
import matplotlib.pyplot as plt
from skimage.color import label2rgb

In [ ]:
size = 400
thresh = 0.3
disk = 3.

In [ ]:
im = process_video('images/cheese_gel.avi', threshold=thresh, size=size, skip_size=239)[1]
F, G = analyse_image(im, L=50)
y = F/G

In [ ]:
def model(thresh, disk):
    noise = generate_noise(size=size, threshold=thresh, disk_size=disk, amount=1)
    F, G = analyse_video(noise, L=50)
    return np.mean(F, axis=0)/np.mean(G, axis=0)

def loss_fn(thresh, disk):
    return np.sum((model(thresh, disk) - y)**2)

In [ ]:
max_iter = 10
step_thresh, step_disk = (0.01, 0.1)
learning_rate = 0.1
loss_fn_arr=[]
thresh_arr=[]
disk_arr=[]
for i in range(max_iter):
    loss = loss_fn(thresh, disk)
    grad1 = (loss_fn(thresh+step_thresh, disk)-loss)/step_thresh
    grad2 = (loss_fn(thresh, disk+step_disk)-loss)/step_disk
    thresh -= grad1 * learning_rate
    disk -= grad2 * learning_rate
    
    loss_fn_arr.append(loss)
    thresh_arr.append(thresh)
    disk_arr.append(disk)


In [ ]:
print(thresh_arr)

In [ ]:
noise = generate_noise(size, threshold=thresh, disk_size=disk, amount=3)
F_noise, G_noise = analyse_video(noise, L=50)
plt.plot(np.mean(F_noise, axis=0)/np.mean(G_noise, axis=0))
plt.show()

plt.imshow(label2rgb(noise[0]))
plt.show()

In [ ]:
plt.plot(F/G)
plt.show()

plt.imshow(label2rgb(im))
plt.show()